# LangChain 1.0 Runtime Examples

This notebook demonstrates using Runtime in **Tools** and **Middleware** based on official LangChain 1.0 documentation.

## Key Concepts:
- **Context**: Static information like user ID, DB connections, API keys
- **Store**: Long-term memory for persisting data across conversations
- **ToolRuntime**: Access to runtime information inside tools
- **Middleware**: Pre/post model hooks and dynamic prompts

## Setup
```bash
pip install langchain langgraph langchain-openai
```

## 1. Import Dependencies

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent, AgentState
from langchain.tools import tool, ToolRuntime
from langchain.messages import AnyMessage
from langchain.agents.middleware import (
    dynamic_prompt, 
    before_model, 
    after_model,
    ModelRequest
)
from langgraph.runtime import Runtime

## 2. Define Context Schema

Using dataclass to define the context structure

In [ ]:
@dataclass
class Context:
    """Context schema with user information"""
    user_id: str
    user_name: str
    database_url: str
    organization_id: str
    api_key: str

# Create a sample context
sample_context = Context(
    user_id="user_12345",
    user_name="John Smith",
    database_url="postgresql://localhost:5432/mydb",
    organization_id="org_789",
    api_key="sk-abc123xyz"
)

print("Context Schema:")
print(sample_context)

## 3. Basic Example: Simple Context Access

In [ ]:
@dataclass
class SimpleContext:
    user_name: str

# This is a minimal example from the docs
agent = create_agent(
    model="openai:gpt-4",
    tools=[],
    context_schema=SimpleContext  
)

# Invoke with context
# result = agent.invoke(
#     {"messages": [{"role": "user", "content": "What's my name?"}]},
#     context=SimpleContext(user_name="John Smith")  
# )

print("Basic agent with simple context created!")

## 4. Tools with ToolRuntime

### Example 1: Accessing Context in Tools

In [ ]:
@tool
def get_user_name(runtime: ToolRuntime[Context]) -> str:
    """Get the current user's name from context."""
    user_name = runtime.context.user_name
    return f"The user's name is {user_name}"

@tool
def get_database_connection(runtime: ToolRuntime[Context]) -> str:
    """Get database connection information from context."""
    db_url = runtime.context.database_url
    org_id = runtime.context.organization_id
    return f"Database URL: {db_url}, Organization: {org_id}"

print("Context access tools created!")
print(f"Tool 1: {get_user_name.name}")
print(f"Tool 2: {get_database_connection.name}")

### Example 2: Using Store for Long-term Memory

In [ ]:
@tool
def fetch_user_email_preferences(runtime: ToolRuntime[Context]) -> str:
    """Fetch the user's email preferences from the store."""
    user_id = runtime.context.user_id
    
    # Default preferences
    preferences: str = "The user prefers you to write a brief and polite email."
    
    # Try to fetch from store if available
    if runtime.store:
        if memory := runtime.store.get(("users",), user_id):
            preferences = memory.value["preferences"]
    
    return preferences

@tool
def save_user_preferences(preferences: str, runtime: ToolRuntime[Context]) -> str:
    """Save user preferences to the store."""
    user_id = runtime.context.user_id
    
    if runtime.store:
        runtime.store.put(
            ("users",), 
            user_id, 
            {"preferences": preferences}
        )
        return f"Saved preferences for user {user_id}"
    
    return "Store not available"

print("Store-based tools created!")

### Example 3: Tool with Multiple Store Operations

In [ ]:
@tool
def save_conversation_summary(summary: str, runtime: ToolRuntime[Context]) -> str:
    """Save a conversation summary to long-term memory."""
    user_id = runtime.context.user_id
    
    if runtime.store:
        namespace = ("conversations", user_id)
        key = f"summary_{runtime.context.organization_id}"
        
        runtime.store.put(namespace, key, {
            "summary": summary,
            "user_name": runtime.context.user_name,
            "timestamp": "2025-10-21"
        })
        
        return f"Saved summary for {runtime.context.user_name}"
    
    return "Store not available"

@tool
def retrieve_past_summaries(runtime: ToolRuntime[Context]) -> str:
    """Retrieve past conversation summaries from long-term memory."""
    user_id = runtime.context.user_id
    
    if runtime.store:
        namespace = ("conversations", user_id)
        items = runtime.store.search(namespace, limit=5)
        
        summaries = [item.value.get("summary") for item in items]
        return f"Found {len(summaries)} summaries: {summaries}"
    
    return "Store not available"

print("Conversation tools created!")

### Example 4: Tool with API Key Access

In [ ]:
@tool
def call_external_api(endpoint: str, runtime: ToolRuntime[Context]) -> str:
    """Call an external API using the API key from context."""
    api_key = runtime.context.api_key
    org_id = runtime.context.organization_id
    user_name = runtime.context.user_name
    
    # Simulate API call
    print(f"Calling API: {endpoint}")
    print(f"Organization: {org_id}")
    print(f"API Key: {api_key[:8]}...")
    print(f"User: {user_name}")
    
    return f"API call to {endpoint} successful for {user_name}"

print("API tool created!")

## 5. Middleware: Dynamic Prompts

In [ ]:
@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    """Create a dynamic system prompt based on user context."""
    user_name = request.runtime.context.user_name
    system_prompt = f"You are a helpful assistant. Address the user as {user_name}."
    return system_prompt

@dynamic_prompt
def personalized_greeting_prompt(request: ModelRequest) -> str:
    """Add personalized greeting based on organization."""
    user_name = request.runtime.context.user_name
    org_id = request.runtime.context.organization_id
    return f"Hello {user_name} from {org_id}! How can I help you today?"

print("Dynamic prompt functions created!")

## 6. Middleware: Before Model Hooks

In [ ]:
@before_model
def log_before_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    """Log information before the model is called."""
    user_name = runtime.context.user_name
    user_id = runtime.context.user_id
    
    print(f"\n{'='*50}")
    print(f"BEFORE MODEL: Processing request for user: {user_name} (ID: {user_id})")
    print(f"Message count: {len(state.get('messages', []))}")
    print(f"{'='*50}")
    
    return None

@before_model
def check_rate_limit(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    """Check rate limits before calling the model."""
    user_id = runtime.context.user_id
    
    if runtime.store:
        namespace = ("rate_limits", user_id)
        rate_data = runtime.store.get(namespace, "requests")
        
        if rate_data:
            count = rate_data.value.get("count", 0)
            if count > 100:
                raise Exception(f"Rate limit exceeded for user {user_id}")
            runtime.store.put(namespace, "requests", {"count": count + 1})
        else:
            runtime.store.put(namespace, "requests", {"count": 1})
    
    print(f"Rate limit check passed for user {user_id}")
    return None

print("Before model hooks created!")

## 7. Middleware: After Model Hooks

In [ ]:
@after_model
def log_after_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    """Log information after the model generates a response."""
    user_name = runtime.context.user_name
    
    print(f"\n{'='*50}")
    print(f"AFTER MODEL: Completed request for user: {user_name}")
    print(f"{'='*50}\n")
    
    return None

@after_model
def save_response_to_store(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    """Save the model response to long-term memory."""
    user_id = runtime.context.user_id
    
    if runtime.store:
        namespace = ("model_responses", user_id)
        messages = state.get("messages", [])
        
        if messages:
            last_message = str(messages[-1])[:200]
            runtime.store.put(namespace, "last_response", {
                "content": last_message,
                "timestamp": "2025-10-21",
                "user_name": runtime.context.user_name
            })
    
    return None

print("After model hooks created!")

## 8. Complete Example: Agent with All Components

In [ ]:
# Collect all tools
all_tools = [
    get_user_name,
    get_database_connection,
    fetch_user_email_preferences,
    save_user_preferences,
    save_conversation_summary,
    retrieve_past_summaries,
    call_external_api
]

# Collect all middleware
all_middleware = [
    dynamic_system_prompt,
    log_before_model,
    check_rate_limit,
    log_after_model,
    save_response_to_store
]

# Create the agent
agent = create_agent(
    model="openai:gpt-4",
    tools=all_tools,
    middleware=all_middleware,
    context_schema=Context
)

print("Complete agent with all tools and middleware created!")
print(f"Tools: {len(all_tools)}")
print(f"Middleware: {len(all_middleware)}")

## 9. Invoke the Agent

In [ ]:
# Set up your OpenAI API key first
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Create context
context = Context(
    user_id="user_12345",
    user_name="John Smith",
    database_url="postgresql://localhost:5432/mydb",
    organization_id="org_789",
    api_key="sk-abc123xyz"
)

# Invoke the agent
# result = agent.invoke(
#     {"messages": [{"role": "user", "content": "What's my name and can you check my preferences?"}]},
#     context=context
# )

# print("\nAgent Response:")
# print(result)

print("Agent is ready to invoke!")
print("Uncomment the code above and set your OpenAI API key to test.")

## 10. Example: Different Contexts for Different Users

In [ ]:
# User 1
context_user1 = Context(
    user_id="user_001",
    user_name="Alice Johnson",
    database_url="postgresql://localhost:5432/alice_db",
    organization_id="org_alpha",
    api_key="sk-alice123"
)

# User 2
context_user2 = Context(
    user_id="user_002",
    user_name="Bob Williams",
    database_url="postgresql://localhost:5432/bob_db",
    organization_id="org_beta",
    api_key="sk-bob456"
)

# Same agent, different contexts
# result1 = agent.invoke(
#     {"messages": [{"role": "user", "content": "What's my name?"}]},
#     context=context_user1
# )

# result2 = agent.invoke(
#     {"messages": [{"role": "user", "content": "What's my name?"}]},
#     context=context_user2
# )

print("Multiple user contexts created!")
print(f"User 1: {context_user1.user_name}")
print(f"User 2: {context_user2.user_name}")

## 11. Minimal Working Example from Documentation

In [ ]:
# This is the exact example from the docs
@dataclass
class MinimalContext:
    user_name: str

@dynamic_prompt
def minimal_system_prompt(request: ModelRequest) -> str:
    user_name = request.runtime.context.user_name
    return f"You are a helpful assistant. Address the user as {user_name}."

@before_model
def minimal_log_before(state: AgentState, runtime: Runtime[MinimalContext]) -> dict | None:
    print(f"Processing request for user: {runtime.context.user_name}")
    return None

@after_model
def minimal_log_after(state: AgentState, runtime: Runtime[MinimalContext]) -> dict | None:
    print(f"Completed request for user: {runtime.context.user_name}")
    return None

minimal_agent = create_agent(
    model="openai:gpt-4",
    tools=[],
    middleware=[minimal_system_prompt, minimal_log_before, minimal_log_after],
    context_schema=MinimalContext
)

# Invoke
# result = minimal_agent.invoke(
#     {"messages": [{"role": "user", "content": "What's my name?"}]},
#     context=MinimalContext(user_name="John Smith")
# )

print("Minimal example from docs created!")

## Summary

This notebook demonstrated LangChain 1.0 Runtime patterns from the official documentation:

### **Tools with ToolRuntime:**
```python
@tool
def my_tool(runtime: ToolRuntime[Context]) -> str:
    user_id = runtime.context.user_id
    if runtime.store:
        data = runtime.store.get(namespace, key)
    return result
```

### **Middleware Decorators:**
```python
@dynamic_prompt
def my_prompt(request: ModelRequest) -> str:
    return f"Hello {request.runtime.context.user_name}"

@before_model
def before_hook(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    # Pre-processing
    return None

@after_model
def after_hook(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    # Post-processing
    return None
```

### **Agent Creation:**
```python
agent = create_agent(
    model="openai:gpt-4",
    tools=[...],
    middleware=[...],
    context_schema=Context
)
```

### **Invocation:**
```python
result = agent.invoke(
    {"messages": [{"role": "user", "content": "..."}]},
    context=Context(...)
)
```